<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


## Adversarial Search (25 Points)

HW3 :: Practical Question 1

<br>

Corresponding TA: Aryan Ahadinia, Saeed Hedayatiyan


Consider the given $8\times8$ board game. There are two players in this game and they try to block each other and limit their valid moves by making walls. Their moves are limited to the 8 cells (or less when they are around the edges) beside them and the walls will start to appear only when a player steps into a random generated yellow cell and the wall will continue to be built until the player reaches the next yellow cell. The players start the game at $(0, 0)$ and $(7, 7)$ and they keep playing until one of them loses. There are multiple ways for a player to win the match:

- If a player steps into a wall or out of the board, the player loses the match!
- If the players reach the maximum moves the game finishes and the winner would be chosen based on the maximum valid move they have left plus the number of walls they have made and "No more moves" massege will appear on terminal.
- If the getMove() function does not return any value after 2 seconds, the player loses the match and "Time limit exceeded" massage will appear on terminal.

<center>
<img src="assets/Q3.png" style="width: 50%"/>
</center>


In this question, the codes for starting the game were provided for you in Helper-codes folder. You are expected to implement the MinimaxPlayer class (which inherit the Player class) with alpha-beta pruning.

In [1]:
%pip install tk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import random
import time

from Helper_codes.graphics import *
from Helper_codes.question3 import Cell
from Helper_codes.question3 import IntPair
from Helper_codes.question3 import Player
from Helper_codes.question3 import NaivePlayer
from Helper_codes.question3 import Board
from Helper_codes.question3 import Game


You can run cell below and watch two naive players.

In [2]:
p1 = NaivePlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

numberOfMatches = 1
score1, score2 = g.start(numberOfMatches)

print(score1 / numberOfMatches)


Player 2 has exceeded the time limit
Player 1 has won

1.0


In [39]:
from copy import deepcopy
import time
class MinimaxPlayer(Player):
    def __init__(self, col, x, y, max_depth=4):
        super().__init__(col, x, y)
        self.max_depth = max_depth
        self.t0=None

    def get_neighbours_coordinates(self,board,player_col):
        x=board.getPlayerX(player_col)
        y=board.getPlayerY(player_col)
        rival_col=1 if player_col==2 else 1
        neighbours_coordinates=[]
        for i in [-1,0,1]:
            for j in [-1,0,1]:
                if i==j==0:continue
                x_neighbour=x+i
                y_neighbour=y+j
                if 0<=x_neighbour<board.getSize() and 0<=y_neighbour<board.getSize() \
                and board.getCell(x_neighbour,y_neighbour).getColor() not in [1,2] \
                and not(x_neighbour == board.getPlayerX(rival_col) and y_neighbour==board.getPlayerY(rival_col)):
                    neighbours_coordinates.append(IntPair(x_neighbour,y_neighbour))
        return neighbours_coordinates
    def min_value(self, board, alpha, beta, depth):
        rival_col=1 if self.getCol()==2 else 2
        neighbours=self.get_neighbours_coordinates(board,rival_col)
        if depth>=self.max_depth or not neighbours:
            return board.getScore(self.getCol())-board.getScore(rival_col),IntPair(board.getPlayerX(rival_col),board.getPlayerY(rival_col))
        
        if time.time()-self.t0>2:
            raise Exception('maximum time reached!')
        best_val=float('inf')
        best_move=None
        
        neighbours=self.get_neighbours_coordinates(board,rival_col)
        for coordinate in neighbours:
            copied_board=deepcopy(board)
            copied_board.move(coordinate,rival_col)
            val=self.max_value(copied_board,alpha,beta,depth+1)[0]
            if val<best_val:
                best_val=val
                best_move=coordinate
            beta=min(beta,val)
            if beta<=alpha:
                break
        return best_val,best_move

    def max_value(self, board, alpha, beta, depth):
        temp_x,temp_y=self.getX(),self.getY()
        neighbours=self.get_neighbours_coordinates(board,self.getCol())
        rival_col=1 if self.getCol()==2 else 2
        if depth>=self.max_depth or not neighbours:
            return board.getScore(self.getCol())-board.getScore(rival_col),IntPair(self.getX(),self.getY())
        
        if time.time()-self.t0>2:
            raise Exception('maximum time reached!')
        best_val=float('-inf')
        best_move=None
        neighbours=self.get_neighbours_coordinates(board,self.getCol())
        for coordinate in neighbours:
            copied_board=deepcopy(board)
            copied_board.move(coordinate,self.getCol())
            self.setX(coordinate.x)
            self.setY(coordinate.y)
            val=self.min_value(copied_board,alpha,beta,depth+1)[0]
            self.setX(temp_x)
            self.setY(temp_y)
            if val>best_val:
                best_val=val
                best_move=coordinate
            alpha=max(alpha,val)
            if beta<=alpha:
                break
        return best_val,best_move

    def getMove(self, board):
        self.t0=time.time()
        if board.getNumberOfMoves() > board.maxNumberOfMoves:
            return IntPair(20, 20)
        if not self.get_neighbours_coordinates(board,self.getCol()):
            return IntPair(-10, -10)
        try:
            neighbour = self.max_value(board, float('-inf'),float('inf'), 0)[1]
            return neighbour
        except:
            return IntPair(-10, -10)
        


In [40]:
p1 = MinimaxPlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

number_of_matches = 1
score1, score2 = g.start(number_of_matches)

print(score1 / number_of_matches)


No more moves!
Player 1 has won
score player 1: 36
score player 2: 19

1.0
